# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
# импортирую библиотеку pandas как главного инструмента для анализа структурных данных и временных рядов
import pandas as pd
# импортирую библиотеку seaborn как главного инструмента для визуализации
import seaborn as sns
# импортирую библиотеку matplotlib для работы с двумерными графиками
import matplotlib.pyplot as plt
# выберу стиль для plt инструментов
plt.style.use('ggplot')
# импортирую библиотеку scipy для выполнения математических, научных и инженерных расчётов
from scipy import stats as st
# импортирую библиотеку для работы со временем
from datetime import datetime
# импортирую библиотеку для работы с массивами данных. NumPy - основополагающая библиотека, необходимая для научных вычислений
import numpy as np
# импортирую библиотеку и инструменты Scikit-learn - интегратора классических алгоритмов машинного обучения
# алгоритм классификации дерева решений
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
# алгоритм классификации случайный лес
from sklearn.ensemble import RandomForestClassifier
# алгоритм классификации логическая регрессия
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score


In [2]:
# открываем файл
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
# начинаем изучать файл
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
# получим общую информацию о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


**Описание данных:**

*Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:*
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

**Анализируя полученную информацию устанавливаем следующее описание датафрейма:**

*Общая информация по фрейму данных: в нашем распоржении фрейм данных, состоящий из  5 столбцов и 3214 строк. Из 5 столбцов 4 - имеют формат вещественных чисел, 1 столбец - бинарный, имеющий значение 0 или 1. Предобработка данных не требуется, датасет готов к следующему этапу работы*.

## Разбейте данные на выборки

In [5]:
# псмотрим на бинарную разбивку по классам
df['is_ultra'].value_counts().head()

0    2229
1     985
Name: is_ultra, dtype: int64

**Идея, лежащая в основании машинного обучения, заключается в построении модели обучения с учителем (контролируемое) методами классификации. Построение системы, способной проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра», предполагает проведение бинарной классификации при наличии 3214 образцов (совокупности признаков), из них Ультра» - 985, «Смарт» - 2229. В рассматриваемом фрейме данных 985 образцов маркированы как класс - Ультра, 2229 образцов имеют маркировку Смарт. В предлагаемом сценарии условия задачи наш фрейм данных является четырехмерным, то есть каждый образец имеет четыре связанных с ним признака (атрибуты, данные измерений, размерности) - x1, x2, x3, x4. Целевой признак в нашей задаче категориальный -  is_ultra.**

*Как начающий специалист Data Science выдвигаю свое первое предположение, как устроены взаимосвязи датафрейма.
Предполагаю, что если пользователь:*
- укладывается в 500 минут разговора - ДА - 50 сообщений - ДА - 15 Гб интернет-трафика - ДА он выберет тариф «Смарт».
- укладыевается выше 500 мин и до  3000 минут разговора - ДА - 1000 сообщений - ДА - 30 Гб интернет-трафика - ДА он выберет «Ультра».

**Как видим данное предположение хорошо ложится в модель решающего дерева, которое определяет тариф. Чтобы найти то самое дерево, нужно обучить модель: подобрать алгоритм, который больше всего подойдёт нашей обучающей выборке. За это отвечают алгоритмы обучения, а их результат работы называется обученной моделью.**

In [6]:
# каждой модели в sklearn соответствует отдельная структура данных
#  создаём объект структуры данных DecisionTreeClassifier() 
#model = DecisionTreeClassifier(random_state=12345)

In [7]:
#model.fit(features, target)

In [8]:
# возьмем первые 5 строк данных для проведения теста модели
#test_df=pd.read_csv('/datasets/users_behavior.csv', nrows=5)

In [9]:
#test_features = test_df.drop(['is_ultra'], axis=1)
#test_target = test_df['is_ultra']

In [10]:
#test_predictions = model.predict(test_features)

In [11]:
#print('Предсказания:',test_predictions)
#print('Правильные ответы:', test_target.to_numpy())

In [12]:
# возьмем первые 5 строк данных для проведения теста модели
#test_df_all = pd.read_csv('/datasets/users_behavior.csv', nrows=5)

In [13]:
# возьмем весь фрейм данных
#test_features_all = test_df_all.drop(['is_ultra'], axis=1)
#test_target_all = test_df_all['is_ultra']

In [14]:
# построим функцию для выявления ошибок по всему фрейму данных
#def error_count(answers, predictions):
    #count = 0
    #for i in range(len(answers)):
       # if answers[i] != predictions[i]:
            #count += 1
   # return count

#print("Ошибок:", error_count(test_target_all, test_predictions))

In [15]:
# # построим функцию для выявления качества модели
#def accuracy(answers, predictions):
    #correct = 0 
    #for i in range(len(answers)):
        #if answers[i] == predictions[i]:
            # correct += 1
    #return correct / len(answers)

#print("Accuracy:", accuracy(test_target_all, test_predictions))

## Исследуйте модели

**Начнем с модели решающее дерево**

In [16]:
# делим данные на обучающую и валидационную выборки
# df_train, df_valid = train_test_split(df, test_size=0.25, random_state=12345)

In [17]:
# создаю выборки для обучения модели и ее тестирования 
df_train, df_test = train_test_split(df, test_size=0.40, random_state=12345)

In [18]:
# создаю отдельно выборку для валидации в тестовой выборке мы забираем на валидацию половину 50, получается - 20 как планировали
df_test, df_valid = train_test_split(df_test, test_size=0.50, random_state=12345)

In [19]:
# посмотрим на количество строк и столбцов в обучающем наборе данных
df_train.shape

(1928, 5)

In [20]:
# посмотрим на количество строк и столбцов в тестирующем наборе данных
df_test.shape

(643, 5)

In [21]:
# посмотрим на количество строк и столбцов в валидационном (проверочном) наборе данных
df_valid.shape

(643, 5)

In [22]:
# разбиваем данные на выборки
# создаем отдельно фичи
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

**модель: Решающее дерево (decision tree, далее - model_dt)**

In [23]:
# создаем цикл для max_depth от 1 до 5
for max_depth in range(1,6):
    model_dt = DecisionTreeClassifier(random_state=12345, max_depth=max_depth) 
# обучаем модель с заданной глубиной дерева
    model_dt.fit(features_train, target_train) # обучаем модель
    predictions = model_dt.predict(features_valid) # получаем предсказания модели
    result = accuracy_score(target_valid, predictions) # посчитаем качество модели
    print('max_depth =',max_depth,':',result)


max_depth = 1 : 0.7356143079315708
max_depth = 2 : 0.7744945567651633
max_depth = 3 : 0.7791601866251944
max_depth = 4 : 0.7744945567651633
max_depth = 5 : 0.7838258164852255


**Попробуем новый алгоритм классификации — случайный лес (random forest, далее - model_rf)**

In [24]:
# отделим 25% данных для валидационной выборки
#df_train_rf, df_vali_rf = train_test_split(df, test_size=0.25, random_state=12345) 
#features_train_rf = df_train_rf.drop(['is_ultra'], axis=1)
#features_valid_rf = df_valid.drop(['is_ultra'], axis=1)
#target_valid_rf = df_valid['is_ultra']

best_model_rf = None
best_result_rf = 0
for est in range(1, 11):
    model_rf = RandomForestClassifier(random_state=12345, n_estimators=est) # обучим модель с заданным количеством деревьев
    model_rf.fit(features_train, target_train) # обучим модель на тренировочной выборке
    result_rf = model_rf.score(features_valid, target_valid) # посчитаем качество модели на валидационной выборке
    if result_rf > best_result_rf:
        best_model_rf = model_rf # сохраним наилучшую модель
        best_result_rf = result_rf #  сохраним наилучшее значение метрики accuracy на валидационных данных

print("Accuracy наилучшей модели на валидационной выборке:", best_result_rf)

Accuracy наилучшей модели на валидационной выборке: 0.7869362363919129


**Попробуем ещё один алгоритм — логистическую регрессию**

In [25]:
#df_train_lr, df_valid_lr = train_test_split(df, test_size=0.25, random_state=12345) # отделите 25% данных для валидаионной выборки

#features_train_lr = df_train_lr.drop(['is_ultra'], axis=1)
#target_train_lr = df_train_lr['is_ultra']
#features_valid_lr = df_valid.drop(['is_ultra'], axis=1)
#target_valid_lr = df_valid['is_ultra']

model_lr = LogisticRegression(random_state=12345) # инициализируйте модель логистической регрессии с параметром random_state=12345
model_lr.fit(features_train, target_train) # обучите модель на тренировочной выборке
result_lr = model_lr.score(features_valid, target_valid) # получите метрику качества модели на валидационной выборке

print("Accuracy модели логистической регрессии на валидационной выборке:", result_lr)

Accuracy модели логистической регрессии на валидационной выборке: 0.7402799377916018


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


**Проведем сравнение моделей:**
- максимально полученный Accuracy на выобрке в алгоритме классификации реашающее дерево -  0.7838258164852255
- Accuracy на выобрке в алгоритме классификации случайный лес - 0.7869362363919129
- Accuracy модели логистической регрессии на валидационной выборке: 0.7402799377916018

**Вывод: Самое высокое качество Accuracy на выборке в алгоритме классификации случайный лес - 0.7869362363919129**


## Проверьте модель на тестовой выборке

In [26]:
#test_df = pd.read_csv('/datasets/users_behavior.csv')
#features = df.drop(['is_ultra'], axis=1)
#target = df['is_ultra']



#model_test = RandomForestClassifier(random_state=12345)
#model_test.fit(features, target)
#test_features = test_df.drop(['is_ultra'], axis=1)
#test_target = test_df['is_ultra']
#test_predictions = model.predict(test_features)


#def error_count(answers, predictions):
    #count = 0
    #for i in range(len(answers)):
       # if answers[i] != predictions[i]:
         #   count += 1
  #  return count

#def accuracy(answers, predictions):
    #correct = 0
    #for i in range(len(answers)):
        #if answers[i] == predictions[i]:
            #correct += 1
    #return correct / len(answers)

#print("Accuracy:", accuracy(test_target, test_predictions))

**Проверка accuracy на тестовой выборке показывает результат выше 0.75** 

**начну тестировать model_dt, решающего дерева**

In [27]:
# готовим исключительно признаки на тестовой выборке
features_test = df_test.drop(['is_ultra'], axis=1)
# готовим исключительно целевой признак на тестовой выборке
target_test = df_test['is_ultra']

In [28]:
# точность на тестовой выборке
model_dt = DecisionTreeClassifier(random_state=12345, max_depth=max_depth)
model_dt.fit(features_train, target_train)
model_dt.predict(features_test)
model_dt.score(features_test, target_test)

0.7791601866251944

<div class="alert alert-block alert-warning">
<b>Комментарий:</b> точность на тесте модели 0.7791601866251944

**тестирую случайный лес (random forest, далее - model_rf)**

In [29]:
# точность при тестировании random forest
model_rf = RandomForestClassifier(random_state=12345, max_depth=max_depth)
model_rf.fit(features_train, target_train)
model_rf.predict(features_test)
model_rf.score(features_test, target_test)

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.7931570762052877

<div class="alert alert-block alert-warning">
<b>Комментарий:</b> точность на тесте модели RF = 0.7931570762052877.

**Провожу тестирование с моделью логистическая регрессия**

In [30]:
result_lr = model_lr.score(features_test, target_test)

print("Accuracy модели логистической регрессии на тестовой выборке:", result_lr)

Accuracy модели логистической регрессии на тестовой выборке: 0.7589424572317263


**Вывод:**

*случайный лес (random forest) продемонстрировал на тестовой выборке наибольшее Accuracy модели 0.7931570762052877*

## (бонус) Проверьте модели на адекватность

*в тренажере обучения проверку класификации модели на адекватность не установил*

In [31]:
# определяю больший класс
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

In [32]:
# общее количество данных
df.shape[0]

3214

In [33]:
# нахожу константу путем отношения количества наибольшего класса smart - 2229 к общему количеству - 3214
constanta = (df['is_ultra'].value_counts() / df.shape[0]).loc[0]
print(constanta)

0.693528313627878


In [34]:
# строю условие если константа меньше найденного значения лучшей по точности модели, то печатаем модель адекватна
# иначе отправляем модель на доработку
if  constanta < 0.79:
    print('Модель адекватна')
else:
    print('Модель трубует доработки')

Модель адекватна
